# Estimating the aDDM
### Author: Brenden Eum (2023)

This script will read already-transformed data containing trial-by-trial details and fixations and fit the aDDM to this data. It uses the Tavares Toolbox (Tavares et al. 2017) rewritten for Julia by Lynn in Summer 2023. See https://github.com/aDDM-Toolbox/ADDM.jl for the toolbox. 

I've made a changes to the toolbox in order to suit my project. Some of these are small changes, like changing object types, saving outputs to csv files, or changing the grid for grid search. I don't document small changes since they'll probably change over multiple iterations. I do document major changes to the toolbox: (1) The toolbox now fits the model to the output of load_data_from_csv instead of simulating data; (2) The toolbox no longer throws a domain error when $\theta \notin [0,1]$.

Inputs for this notebook:
- Data
    - data/expdata*.csv:    Data pertaining to the details of each trial. An observation is a subject-trial. It must include the following columns [parcode,trial,rt,choice,item_left,item_right], which correspond to [subject, trial, response time in ms, choice (-1 left, 1 right), value of left item, value of right item] respectively.
    - data/fixations*.csv:  Data pertaining to the fixations within each trial. An observation is a subject-trial-fixation. It must include columns [parcode,trial,fix_item,fix_time], which correspond to [subject, trial, fixation location (1 left, 2 right, otherwise other), fixation duration in ms].
- Code
    - src/ addm_grid_search.jl: Takes a grid of parameters, makes every combination, and calculates negative log likelihood (NLL) of aDDM for every combo given some data.
    - addm.jl: Defines aDDM class and functions for calculating trial likelihood, summing likelihoods, and simulating.
    - ddm.jl: Defines DDM class and functions for calculating trial likelihood, summing likelihoods, and simulating.
    - util.jl: Contains functions for prepping data for aDDM fitting and estimating empirical fixation properties.


Outputs for this notebook:
- GainFit.csv:  aDDM estimates for the gain condition. Estimates are for subjects included in the input datasets.
- LossFit.csv:  aDDM estimates for the loss condition.

# Preamble

A set of standard things I like to do before getting started with any notebook.

In [1]:
import Pkg;
Pkg.add("DataFrames");
Pkg.add("CSV");
using DataFrames;
using CSV;

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


# Activate the Environment

Typically this happens by

```
using Pkg
Pkg.activate("addm")
```

**Note that single vs. double quotes matter.**

But this does not work as expected in a notebook unless the `Project.toml` is in the same folder as the notebook file.

In [2]:
pwd();
Base.load_path();
include("../../src/addm_grid_search.jl");
Base.load_path();

  Activating project at `~/Toolboxes/ADDM.jl/docs/src/addm`
  Activating project at `~/Toolboxes/ADDM.jl/docs/src/addm`
  Activating project at `~/Toolboxes/ADDM.jl/docs/src/addm`
  Activating project at `~/Toolboxes/ADDM.jl/docs/src/addm`
  Activating project at `~/Toolboxes/ADDM.jl/docs/src/addm`
  Activating project at `~/Toolboxes/ADDM.jl/docs/src/addm`


# Fit the aDDM

Let's do a 2-iteration grid search. We'll start with a low resolution grid search, then once we have rough estimates per subject, we can zoom into the region around the rough estimates for a precise measure.

### Prep for gain and loss: load data, placeholders, set step sizes

In [31]:
addm = aDDM(.005, .07, .3); # These can be anything. They exist because you need an aDDM object. (d, s, t).

dataGain = load_data_from_csv("../../data/expdataGain.csv", "../../data/fixationsGain.csv", convertItemValues=nothing)
dataLoss = load_data_from_csv("../../data/expdataLoss.csv", "../../data/fixationsLoss.csv", convertItemValues=nothing);

subjectCount = length(collect(keys(dataGain)))

dStepSize1 = .001
σStepSize1 = .04
θStepSize1 = .2
bStepSize1 = .1   

dStepSize2 = .0005
σStepSize2 = .02
θStepSize2 = .1
bStepSize2 = .05

dStepSize3 = .0001
σStepSize3 = .01
θStepSize3 = .01
bStepSize3 = .01;

dInitialGrid = Any[]
σInitialGrid = Any[]
θInitialGrid = Any[]
bInitialGrid = Any[]
for subject in collect(keys(dataGain))
    push!(dInitialGrid, [0:dStepSize1:.006;])   
    push!(σInitialGrid, [0.01:σStepSize1:.09;])    
    push!(θInitialGrid, [0:θStepSize1:2;])    
    push!(bInitialGrid, [-.1:bStepSize1:.1;]) 
end


### Function: Grid search and report data frame

In [34]:
function fit_aDDM_custom_resolution(addm::aDDM, data::Dict{String, Vector{aDDMTrial}}, dGrid::Vector{Any}, σGrid::Vector{Any}, θGrid::Vector{Any}, bGrid::Vector{Any}, subjectCount::Number)
    """
    """

    print("Approx how many likelihoods to calculate per subject: ")
    print(length(dGrid[1])*length(σGrid[1])*length(θGrid[1])*length(bGrid[1]))

    dList = Vector{Float64}(undef, subjectCount)
    σList = Vector{Float64}(undef, subjectCount)
    θList = Vector{Float64}(undef, subjectCount)
    bList = Vector{Float64}(undef, subjectCount)
    NLLsList = Vector{Float64}(undef, subjectCount);

    ind = 1
    @showprogress for subject in collect(keys(data))

        dEst, σEst, θEst, bEst, NLL_Indiv = aDDM_grid_search(addm, data, dGrid[ind], σGrid[ind], θGrid[ind], bGrid[ind], subject)
    
        dList[ind] = dEst[1]
        σList[ind] = σEst[1]
        θList[ind] = θEst[1]
        bList[ind] = bEst[1]
        NLLsList[ind] = NLL_Indiv
        ind += 1
    
    end

    df = DataFrame(
        subject = collect(keys(data)),
        d = dList,
        s = σList,
        t = θList,
        b = bList
    )
    df = sort(df, :subject)
    NLL = sum(NLLsList)
    return df, NLL
end

fit_aDDM_custom_resolution (generic function with 2 methods)

## Gain Data

### Low resolution grid search
40 mins

In [35]:
dfGain_LowRes, NLL = fit_aDDM_custom_resolution(addm, dataGain, dInitialGrid, σInitialGrid, θInitialGrid, bInitialGrid, subjectCount)

Approx how many likelihoods to calculate per subject: 693

Progress:  12%|████▉                                    |  ETA: 0:16:39

LoadError: TaskFailedException

[91m    nested task error: [39mInterruptException:
    Stacktrace:
     [1] [0m[1mbroadcasted[22m
    [90m   @[39m [90m./[39m[90m[4mbroadcast.jl:1319[24m[39m[90m [inlined][39m
     [2] [0m[1mbroadcasted[22m[0m[1m([22m::[0mFunction, ::[0mBase.Broadcast.Broadcasted[90m{Base.Broadcast.DefaultArrayStyle{1}, Nothing, typeof(>=), Tuple{StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}, Int64}}[39m, ::[0mBase.Broadcast.Broadcasted[90m{Base.Broadcast.DefaultArrayStyle{1}, Nothing, typeof(<=), Tuple{StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}, Int64}}[39m[0m[1m)[22m
    [90m   @[39m [90mBase.Broadcast[39m [90m./[39m[90m[4mbroadcast.jl:1317[24m[39m
     [3] [0m[1maDDM_get_trial_likelihood[22m[0m[1m([22m[90maddm[39m::[0maDDM, [90mtrial[39m::[0maDDMTrial; [90mtimeStep[39m::[0mFloat64, [90mapproxStateStep[39m::[0mFloat64, [90

In [ ]:
"""
ind = 1

@showprogress for subject in collect(keys(data))

  dEst, σEst, θEst, bEst, NLL = aDDM_grid_search(addm, data, dGridLow, σGridLow, θGridLow, bGridLow, subject)

  dList[ind] = dEst[1]
  σList[ind] = σEst[1]
  θList[ind] = θEst[1]
  bList[ind] = bEst[1]
  NLLsList[ind] = NLL
  ind += 1

end

dfGain_LowRes = DataFrame(
  subject = collect(keys(data)),
  d = dList,
  s = σList,
  t = θList,
  b = bList
)
dfGain_LowRes = sort(dfGain_LowRes, :subject)
NLLGain_LowRes = sum(NLLsList) 
println(NLLGain_LowRes)
dfGain_LowRes
"""

### Medium resolution grid search
1 hr

In [ ]:
d = dfGain_LowRes[(dfGain_LowRes.subject .== subject), "d"][1]
σ = dfGain_LowRes[(dfGain_LowRes.subject .== subject), "s"][1]
θ = dfGain_LowRes[(dfGain_LowRes.subject .== subject), "t"][1]
b = dfGain_LowRes[(dfGain_LowRes.subject .== subject), "b"][1]

dLow = d-(dStepSize1/2);
dHigh = d+(dStepSize1/2);
σLow = max(0.001, σ-(σStepSize1/2));
σHigh = σ+(σStepSize1/2);
θLow = θ-(θStepSize1/2);
θHigh = θ+(θStepSize1/2);
bLow = max(-.99, b-(bStepSize1/2));
bHigh = min(.99, b+(bStepSize1/2));

dGrid = [dLow:dStepSize2:dHigh;]      
σGrid = [σLow:σStepSize2:σHigh;]    
θGrid = [θLow:θStepSize2:θHigh;]           
bGrid = [bLow:bStepSize2:bHigh;]  

dfGain_MedRes, NLL = fit_aDDM_custom_resolution(addm, dataGain, dGrid, σGrid, θGrid, bGrid, subjectCount)

In [ ]:
"""
ind = 1

@showprogress for subject in collect(keys(data))

    

    
    
    print("How many likelihoods to calculate: ")
    print(length(dGrid)*length(σGrid)*length(θGrid)*length(bGrid))

    dEst, σEst, θEst, bEst, NLL = aDDM_grid_search(addm, data, dGrid, σGrid, θGrid, bGrid, subject)
  
    dList[ind] = dEst[1]
    σList[ind] = σEst[1]
    θList[ind] = θEst[1]
    bList[ind] = bEst[1]
    NLLsList[ind] = NLL
    ind += 1
  
  end

dfGain_MedRes = DataFrame(
  subject = collect(keys(data)),
  d = dList,
  s = σList,
  t = θList,
  b = bList
)
dfGain_MedRes = sort(dfGain_MedRes, :subject)
NLLGain_MedRes = sum(NLLsList)
println(NLLGain_MedRes)
dfGain_MedRes
"""

### High resolution grid search
12 mins

In [ ]:
d = dfGain_MedRes[(dfGain_MedRes.subject .== subject), "d"][1]
σ = dfGain_MedRes[(dfGain_MedRes.subject .== subject), "s"][1]
θ = dfGain_MedRes[(dfGain_MedRes.subject .== subject), "t"][1]
b = dfGain_MedRes[(dfGain_MedRes.subject .== subject), "b"][1]

dLow = d-(dStepSize2/2);
dHigh = d+(dStepSize2/2);
σLow = max(0.001, σ-(σStepSize2/2));
σHigh = σ+(σStepSize2/2);
θLow = θ-(θStepSize2/2);
θHigh = θ+(θStepSize2/2);
bLow = max(-.99, b-(bStepSize2/2));
bHigh = min(.99, b+(bStepSize2/2));

dGrid = [dLow:dStepSize3:dHigh;]      
σGrid = [σLow:σStepSize3:σHigh;]    
θGrid = [θLow:θStepSize3:θHigh;]           
bGrid = [bLow:bStepSize3:bHigh;] 

dfGain_HiRes, NLL = fit_aDDM_custom_resolution(addm, dataGain, dGrid, σGrid, θGrid, bGrid, subjectCount)

In [ ]:
"""
ind = 1

@showprogress for subject in collect(keys(data))

    d = dfGain_MedRes[(dfGain_MedRes.subject .== subject), "d"][1]
    σ = dfGain_MedRes[(dfGain_MedRes.subject .== subject), "s"][1]
    θ = dfGain_MedRes[(dfGain_MedRes.subject .== subject), "t"][1]
    b = dfGain_MedRes[(dfGain_MedRes.subject .== subject), "b"][1]

    dLow = d-(dStepSize2/2);
    dHigh = d+(dStepSize2/2);
    σLow = max(0.001, σ-(σStepSize2/2));
    σHigh = σ+(σStepSize2/2);
    θLow = θ-(θStepSize2/2);
    θHigh = θ+(θStepSize2/2);
    bLow = max(-.99, b-(bStepSize2/2));
    bHigh = min(.99, b+(bStepSize2/2));

    dGrid = [dLow:dStepSize3:dHigh;]      
    σGrid = [σLow:σStepSize3:σHigh;]    
    θGrid = [θLow:θStepSize3:θHigh;]           
    bGrid = [bLow:bStepSize3:bHigh;] 

    print("How many likelihoods to calculate: ")
    print(length(dGrid)*length(σGrid)*length(θGrid)*length(bGrid))

    dEst, σEst, θEst, bEst, NLL = aDDM_grid_search(addm, data, dGrid, σGrid, θGrid, bGrid, subject)
  
    dList[ind] = dEst[1]
    σList[ind] = σEst[1]
    θList[ind] = θEst[1]
    bList[ind] = bEst[1]
    NLLsList[ind] = NLL
    ind += 1
  
  end

dfGain_HiRes = DataFrame(
  subject = collect(keys(data)),
  d = round.(dList; digits=4),
  s = round.(σList; digits=2),
  t = round.(θList; digits=2),
  b = round.(bList; digits=2)
)
dfGain_HiRes = sort(dfGain_HiRes, :subject)
NLLGain_HiRes = sum(NLLsList)
println(NLLGain_HiRes)
CSV.write("GainFit.csv", dfGain_HiRes)
dfGain_HiRes
"""

## Fit Loss Data

### Low resolution grid search
30 mins

In [9]:
dfLoss_LowRes, NLL = fit_aDDM_custom_resolution(addm, dataLoss, dInitialGrid, σInitialGrid, θInitialGrid, bInitialGrid, subjectCount)

How many likelihoods to calculate: 693

Progress: 100%|█████████████████████████████████████████| Time: 0:25:55


(25×5 DataFrame
 Row │ subject  d        s        t        b       
     │ String   Float64  Float64  Float64  Float64 
─────┼─────────────────────────────────────────────
   1 │ 201        0.001     0.05      1.8      0.0
   2 │ 202        0.001     0.05      2.0      0.1
   3 │ 204        0.002     0.05      1.2      0.0
   4 │ 205        0.0       0.09      0.0      0.0
   5 │ 206        0.002     0.05      1.4      0.1
   6 │ 207        0.001     0.05      0.8      0.0
   7 │ 208        0.002     0.09      2.0      0.0
   8 │ 209        0.001     0.05      1.6      0.0
   9 │ 211        0.001     0.05      1.8      0.0
  10 │ 213        0.002     0.09      1.6      0.0
  11 │ 214        0.0       0.05      0.0      0.0
  ⋮  │    ⋮        ⋮        ⋮        ⋮        ⋮
  16 │ 220        0.002     0.05      1.2      0.1
  17 │ 222        0.001     0.05      1.0      0.0
  18 │ 223        0.002     0.05      1.0      0.0
  19 │ 224        0.003     0.05      1.0      0.0
  20 │ 225     

In [10]:
"""
ind = 1

@showprogress for subject in collect(keys(data))

  dEst, σEst, θEst, bEst, NLL = aDDM_grid_search(addm, data, dGridLow, σGridLow, θGridLow, bGridLow, subject)

  dList[ind] = dEst[1]
  σList[ind] = σEst[1]
  θList[ind] = θEst[1]
  bList[ind] = bEst[1]
  NLLsList[ind] = NLL
  ind += 1

end

dfLoss_LowRes = DataFrame(
  subject = collect(keys(data)),
  d = dList,
  s = σList,
  t = θList,
  b = bList
)
dfLoss_LowRes = sort(dfLoss_LowRes, :subject)
NLLLoss_LowRes = sum(NLLsList) 
println(NLLLoss_LowRes)
dfLoss_LowRes
"""

"ind = 1\n\n@showprogress for subject in collect(keys(data))\n\n  dEst, σEst, θEst, bEst, NLL = aDDM_grid_search(addm, data, dGridLow, σGridLow, θGridLow, bGridLow, subject)\n\n  dList[ind] = dEst[1]\n  σList[ind] = σEst[1]\n  θList[ind] = θEst[1]\n  bList[ind] = bEst[1]\n  NLLsList[ind] = NLL\n  ind += 1\n\nend\n\ndfLoss_LowRes = DataFrame(\n  subject = collect(keys(data)),\n  d = dList,\n  s = σList,\n  t = θList,\n  b = bList\n)\ndfLoss_LowRes = sort(dfLoss_LowRes, :subject)\nNLLLoss_LowRes = sum(NLLsList) \nprintln(NLLLoss_LowRes)\ndfLoss_LowRes\n"

### Medium resolution grid search
1 hr 10 mins

In [37]:
dGrid = Any[]
σGrid = Any[]
θGrid = Any[]
bGrid = Any[]

ind = 1
for subject in collect(keys(dataLoss))

    d = dfLoss_LowRes[(dfLoss_LowRes.subject .== subject), "d"][1]
    σ = dfLoss_LowRes[(dfLoss_LowRes.subject .== subject), "s"][1]
    θ = dfLoss_LowRes[(dfLoss_LowRes.subject .== subject), "t"][1]
    b = dfLoss_LowRes[(dfLoss_LowRes.subject .== subject), "b"][1]

    dLow = d-(dStepSize1/2);
    dHigh = d+(dStepSize1/2);
    σLow = max(0.001, σ-(σStepSize1/2));
    σHigh = σ+(σStepSize1/2);
    θLow = θ-(θStepSize1/2);
    θHigh = θ+(θStepSize1/2);
    bLow = max(-.99, b-(bStepSize1/2));
    bHigh = min(.99, b+(bStepSize1/2));

    push!(dGrid, [dLow:dStepSize2:dHigh;])
    push!(σGrid, [σLow:σStepSize2:σHigh;])    
    push!(θGrid, [θLow:θStepSize2:θHigh;])           
    push!(bGrid, [bLow:bStepSize2:bHigh;])  

    ind += 1

end

dfLoss_MedRes, NLL = fit_aDDM_custom_resolution(addm, dataLoss, dGrid, σGrid, θGrid, bGrid, subjectCount)

Approx how many likelihoods to calculate per subject: 81

Progress: 100%|█████████████████████████████████████████| Time: 0:02:12


(25×5 DataFrame
 Row │ subject  d        s        t        b       
     │ String   Float64  Float64  Float64  Float64 
─────┼─────────────────────────────────────────────
   1 │ 201       0.001      0.05      1.7     0.0
   2 │ 202       0.001      0.05      2.1     0.05
   3 │ 204       0.002      0.05      1.1     0.0
   4 │ 205      -0.0005     0.07     -0.1     0.0
   5 │ 206       0.002      0.05      1.3     0.05
   6 │ 207       0.001      0.05      0.8     0.0
   7 │ 208       0.002      0.07      2.1     0.0
   8 │ 209       0.001      0.05      1.5     0.0
   9 │ 211       0.001      0.05      1.8     0.0
  10 │ 213       0.002      0.07      1.7     0.05
  11 │ 214      -0.0005     0.05     -0.1     0.0
  ⋮  │    ⋮        ⋮        ⋮        ⋮        ⋮
  16 │ 220       0.0015     0.05      1.3     0.05
  17 │ 222       0.001      0.05      1.1     0.0
  18 │ 223       0.0015     0.05      1.1     0.0
  19 │ 224       0.0025     0.07      1.1     0.0
  20 │ 225       0.003    

In [ ]:
"""
ind = 1

@showprogress for subject in collect(keys(data))

    d = dfLoss_LowRes[(dfLoss_LowRes.subject .== subject), "d"][1]
    σ = dfLoss_LowRes[(dfLoss_LowRes.subject .== subject), "s"][1]
    θ = dfLoss_LowRes[(dfLoss_LowRes.subject .== subject), "t"][1]
    b = dfLoss_LowRes[(dfLoss_LowRes.subject .== subject), "b"][1]

    dLow = d-(dStepSize1/2);
    dHigh = d+(dStepSize1/2);
    σLow = max(0.001, σ-(σStepSize1/2));
    σHigh = σ+(σStepSize1/2);
    θLow = θ-(θStepSize1/2);
    θHigh = θ+(θStepSize1/2);
    bLow = max(-.99, b-(bStepSize1/2));
    bHigh = min(.99, b+(bStepSize1/2));

    dGrid = [dLow:dStepSize2:dHigh;]      
    σGrid = [σLow:σStepSize2:σHigh;]    
    θGrid = [θLow:θStepSize2:θHigh;]           
    bGrid = [bLow:bStepSize2:bHigh;]       

    print("How many likelihoods to calculate: ")
    print(length(dGrid)*length(σGrid)*length(θGrid)*length(bGrid))

    dEst, σEst, θEst, bEst, NLL = aDDM_grid_search(addm, data, dGrid, σGrid, θGrid, bGrid, subject)
  
    dList[ind] = dEst[1]
    σList[ind] = σEst[1]
    θList[ind] = θEst[1]
    bList[ind] = bEst[1]
    NLLsList[ind] = NLL
    ind += 1
  
  end

dfLoss_MedRes = DataFrame(
  subject = collect(keys(data)),
  d = dList,
  s = σList,
  t = θList,
  b = bList
)
dfLoss_MedRes = sort(dfLoss_MedRes, :subject)
NLLLoss_MedRes = sum(NLLsList)
println(NLLLoss_MedRes)
dfLoss_MedRes
"""

### High resolution grid search
15 mins

In [38]:
dGrid = Any[]
σGrid = Any[]
θGrid = Any[]
bGrid = Any[]

ind = 1
for subject in collect(keys(dataLoss))

    d = dfLoss_LowRes[(dfLoss_LowRes.subject .== subject), "d"][1]
    σ = dfLoss_LowRes[(dfLoss_LowRes.subject .== subject), "s"][1]
    θ = dfLoss_LowRes[(dfLoss_LowRes.subject .== subject), "t"][1]
    b = dfLoss_LowRes[(dfLoss_LowRes.subject .== subject), "b"][1]

    dLow = d-(dStepSize2/2);
    dHigh = d+(dStepSize2/2);
    σLow = max(0.001, σ-(σStepSize2/2));
    σHigh = σ+(σStepSize2/2);
    θLow = θ-(θStepSize2/2);
    θHigh = θ+(θStepSize2/2);
    bLow = max(-.99, b-(bStepSize2/2));
    bHigh = min(.99, b+(bStepSize2/2));

    push!(dGrid, [dLow:dStepSize3:dHigh;])
    push!(σGrid, [σLow:σStepSize3:σHigh;])    
    push!(θGrid, [θLow:θStepSize3:θHigh;])           
    push!(bGrid, [bLow:bStepSize3:bHigh;])  

    ind += 1

end

dfLoss_HiRes, NLL = fit_aDDM_custom_resolution(addm, dataLoss, dGrid, σGrid, θGrid, bGrid, subjectCount)

Approx how many likelihoods to calculate per subject: 1188

Progress:  32%|█████████████▏                           |  ETA: 0:23:23

LoadError: TaskFailedException

[91m    nested task error: [39mInterruptException:
    Stacktrace:
     [1] [0m[1mbroadcasted[22m
    [90m   @[39m [90m./[39m[90m[4mbroadcast.jl:1319[24m[39m[90m [inlined][39m
     [2] [0m[1mbroadcasted[22m[0m[1m([22m::[0mFunction, ::[0mBase.Broadcast.Broadcasted[90m{Base.Broadcast.DefaultArrayStyle{1}, Nothing, typeof(>=), Tuple{StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}, Int64}}[39m, ::[0mBase.Broadcast.Broadcasted[90m{Base.Broadcast.DefaultArrayStyle{1}, Nothing, typeof(<=), Tuple{StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}, Int64}}[39m[0m[1m)[22m
    [90m   @[39m [90mBase.Broadcast[39m [90m./[39m[90m[4mbroadcast.jl:1317[24m[39m
     [3] [0m[1maDDM_get_trial_likelihood[22m[0m[1m([22m[90maddm[39m::[0maDDM, [90mtrial[39m::[0maDDMTrial; [90mtimeStep[39m::[0mFloat64, [90mapproxStateStep[39m::[0mFloat64, [90

In [ ]:
"""
ind = 1

@showprogress for subject in collect(keys(data))

    d = dfLoss_MedRes[(dfLoss_MedRes.subject .== subject), "d"][1]
    σ = dfLoss_MedRes[(dfLoss_MedRes.subject .== subject), "s"][1]
    θ = dfLoss_MedRes[(dfLoss_MedRes.subject .== subject), "t"][1]
    b = dfLoss_MedRes[(dfLoss_MedRes.subject .== subject), "b"][1]

    dLow = d-(dStepSize2/2);
    dHigh = d+(dStepSize2/2);
    σLow = max(0.001, σ-(σStepSize2/2));
    σHigh = σ+(σStepSize2/2);
    θLow = θ-(θStepSize2/2);
    θHigh = θ+(θStepSize2/2);
    bLow = max(-.99, b-(bStepSize2/2));
    bHigh = min(.99, b+(bStepSize2/2));

    dGrid = [dLow:dStepSize3:dHigh;]      
    σGrid = [σLow:σStepSize3:σHigh;]    
    θGrid = [θLow:θStepSize3:θHigh;]           
    bGrid = [bLow:bStepSize3:bHigh;]  

    print("How many likelihoods to calculate: ")
    print(length(dGrid)*length(σGrid)*length(θGrid)*length(bGrid))

    dEst, σEst, θEst, bEst, NLL = aDDM_grid_search(addm, data, dGrid, σGrid, θGrid, bGrid, subject)
  
    dList[ind] = dEst[1]
    σList[ind] = σEst[1]
    θList[ind] = θEst[1]
    bList[ind] = bEst[1]
    NLLsList[ind] = NLL
    ind += 1
  
  end

dfLoss_HiRes = DataFrame(
  subject = collect(keys(data)),
  d = round.(dList; digits=4),
  s = round.(σList; digits=2),
  t = round.(θList; digits=2),
  b = round.(bList; digits=2)
)
dfLoss_HiRes = sort(dfLoss_HiRes, :subject)
NLLLoss_HiRes = sum(NLLsList)
println(NLLLoss_HiRes)
CSV.write("LossFit.csv", dfLoss_HiRes)
dfLoss_HiRes
"""